In [1]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx
import igraph as ig
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
import pandas as pd
from joblib import Parallel, delayed 
import random
from scipy.linalg import sqrtm
import scipy

In [2]:
def sponge(G):

    selected_edges = [(u,v) for u,v in G.edges if G[u][v]['weight'] < 0]

    G_neg = nx.Graph()
    G_neg.add_nodes_from(G)
    G_neg.add_edges_from(selected_edges)
    degree_neg = [val for (node, val) in G_neg.degree()]
    D_neg = np.diag(degree_neg)
    A_neg = nx.to_numpy_array(G_neg)
    L_neg = D_neg - A_neg

    
    selected_edges = [(u,v) for u,v in G.edges if G[u][v]['weight'] > 0]
    G_pos = nx.Graph()
    G_pos.add_nodes_from(G)
    G_pos.add_edges_from(selected_edges)
    degree_pos = [val for (node, val) in G_pos.degree()]
    D_pos = np.diag(degree_pos)
    A_pos = nx.to_numpy_array(G_pos)
    L_pos = D_pos - A_pos
    
    
    N = L_neg + D_pos
    D, V = scipy.linalg.eigh(N)
    Bs = (V * np.sqrt(D)) @ V.T
    M1 = np.linalg.inv(Bs)
    M2 = L_pos + D_neg
    
    step1 = np.matmul(M1, M2)
    step2 = np.matmul(step1, M1)
    
    return step2

In [3]:
#to find k+1 clusters, use the k smallest eigenvectors of the sponge matrix as the positions 
#apply k-means clustering with n_clust = k+1

#paper: https://arxiv.org/abs/1904.08575

val, vect  = np.linalg.eigh(sponge(G))

positions = vect[:,0:k]

NameError: name 'G' is not defined

In [ ]:
#other method from traag, community spinglass 
#i usually use networkx so just convert to igraph first 
#you can experiment with changing the gamma and gamma.minus params as explained in the documentation 
#https://igraph.org/r/doc/cluster_spinglass.html

h = ig.Graph.from_networkx(G)
ig.summary(h)
communities = h.community_spinglass(weights = "tie", implementation="neg")

clustering = [set() for i in list(set(communities.membership))]
for i in range(len(communities.membership)):
    clustering[communities.membership[i]].add(i)

In [ ]:
plt.hist(communities.membership)